Upload QSignals Test data from S3

In [10]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
bucket='swarmode'
data_key = 'qstrades/qstrades_test3.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data = pd.read_csv(data_location)
pd.read_csv(data_location)

,symbol,daysignal,tradedate,closeprice,openprice,momentum,buyat,sellat,stopat,profit,...,tradesize,ordertype,exitprice,exittime,hitstop,capital,tradeprofit,tradereturn,signalscore,goodsignal
0,WLL,2020-06-22 10:04:08.653154,2020-06-22,1.530,1.50,-0.030000,1.60173,1.73146,1.52587,0.129723,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
1,CDEV,2020-06-22 10:01:57.124039,2020-06-22,1.130,1.16,0.030000,1.24899,1.32576,1.18962,0.076771,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
2,AGS,2020-06-22 10:01:36.013574,2020-06-22,4.030,4.03,0.000000,4.15043,4.40043,4.05043,0.250000,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
3,NERV,2020-06-22 10:03:11.270830,2020-06-22,3.530,3.53,0.000000,3.65481,3.83481,3.55481,0.180000,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
4,OAS,2020-06-22 10:03:16.838265,2020-06-22,0.999,1.04,0.041000,1.10208,1.15367,1.07799,0.051588,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,RARE,2020-06-22 10:03:29.669092,2020-06-22,78.220,77.87,-0.349998,78.90350,79.03730,78.54290,0.133780,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
2210,SOXL,2020-06-22 10:03:45.777411,2020-06-22,181.000,181.90,0.899994,183.62900,183.93700,182.73300,0.307820,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
2211,KPTI,2020-06-22 10:02:53.394201,2020-06-22,19.000,19.58,0.580000,19.90110,19.93350,19.80110,0.032400,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n
2212,CWH,2020-06-22 10:02:10.465100,2020-06-22,27.020,27.15,0.129999,27.46500,27.50980,27.36500,0.044814,...,0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,n


Predict the Qsignals trades dataset

In [11]:
import sagemaker

ep_name = 'goodqsignals-predictor1'

In [12]:
import boto3,sys

sm_rt = boto3.Session().client('runtime.sagemaker')

In [13]:
import numpy as np

train_data, test_data, _ = np.split(data.sample(frac=1, random_state=123), 
                                                  [int(0.01 * len(data)), int(len(data))])  

# Save to CSV files
train_data.to_csv('automl-train.csv', index=False, header=True, sep=',') # Need to keep column names
test_data.to_csv('automl-test.csv', index=False, header=True, sep=',')

In [14]:
%%sh

ls -l *.csv

-rw-r--r-- 1 root root 402869 Jun 22 22:40 automl-test.csv
-rw-r--r-- 1 root root   4155 Jun 22 22:40 automl-train.csv


In [15]:
tp = tn = fp = fn = count = 0

with open ('automl-test.csv') as f:
    lines = f.readlines()
    for l in lines[1:]:   # Skip header
        l = l.split(',')  # Split CSV line into features
        label = l[-1]     # Store 'y'/'n' label
        l = l[:-1]        # Remove label
        l = ','.join(l)   # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))

        if 'y' in label:
            # Sample is positive
            if 'y' in response:
                # True positive
                tp=tp+1
            else:
                # False negative
                fn=fn+1
        else:
            # Sample is negative
            if 'n' in response:
                # True negative
                tn=tn+1
            else:
                # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 Done


Printing Confusion Matrix Values and Key Classifier Metrics

In [16]:
print ("%d %d" % (tn, fp))
print ("%d %d" % (fn, tp))

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tn/(tn+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

1820 0
0 372
1.0000 1.0000 1.0000 1.0000
